In [1]:
from preprocess import dataset,get_Kfold
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from models import FM_MLP,AE_MLP,get_models
from torch import optim
from torch.utils.data import DataLoader,Dataset
import torch 
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import json
import random
from TrainandTest import train_MLP,test_MLP
import pandas as pd

def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
same_seeds(1)

# Config

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
file = './data/totaldata.xlsx'
data = pd.read_excel(file,engine="openpyxl")
data = np.array(data)
para_path = './save/parameter'
batch_size = 300
flag = 0
plot = True
scaler = StandardScaler()

# Train

In [3]:
with open('config.json','r',encoding='utf-8')as f:
    config_list = json.load(f)
total_mse_list = []
for config in config_list:
    Type = config['Type']
    N_size = config['N_size']
    num_epoch = config['num_epoch']
    lr = config['lr']
    decay = config["decay"]
    k = config['k']
    alpha = config['alpha']
    input_size = config['input_size']
    L1,L2 = config['L1'],config['L2']
    
    K_fold_data = get_Kfold(file,k)
    
    for k_step,(trian_data,test_data) in enumerate(K_fold_data):
        tqdm.write(f"-----------[ Times:{flag+1} K:{k_step+1} Type:{Type} N_size:{N_size} ]-----------")
        
        model = get_models(config['model'],input_size,L1,L2).to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(),lr=lr,weight_decay=decay)
        lr_scheduler = optim.lr_scheduler.LambdaLR(optimizer,lambda x: 1 - x/num_epoch,last_epoch=-1)
        
        #get data prepared
        trian_data = dataset(trian_data,Type,N_size,input_size,scaler)
        test_data = dataset(test_data,Type,N_size,input_size,scaler)
        train_loader = DataLoader(dataset=trian_data,batch_size=batch_size)
        test_loader = DataLoader(dataset=test_data,batch_size=batch_size)
        
        #train
        train_MLP(train_loader,
                  model,
                  criterion,
                  alpha,
                  optimizer,
                  num_epoch,
                  k_step,
                  lr_scheduler,
                  para_path,
                  device,
                  flag)
        
        #tese
        mse,rmse = test_MLP(test_loader,
                             model,
                             k_step,
                             criterion,
                             alpha,
 #                            plot,
                             0,
                             para_path,
                             device,
                             flag)
        total_mse_list.append(mse)
    flag +=1
    tqdm.write(f"total rmse of k fold is: {rmse}")

total_rmse = (sum(total_mse_list)/len(data))**0.5
print(f"total rmse of all data is: {total_rmse}")

-----------[ Times:1 K:1 Type:odd N_size:<126 ]-----------
Test Loss: 0.07310, Test RMSE: 0.27038
-----------[ Times:1 K:2 Type:odd N_size:<126 ]-----------
Test Loss: 0.06460, Test RMSE: 0.25417
-----------[ Times:1 K:3 Type:odd N_size:<126 ]-----------
Test Loss: 0.01888, Test RMSE: 0.13740
-----------[ Times:1 K:4 Type:odd N_size:<126 ]-----------
Test Loss: 0.05853, Test RMSE: 0.24193
-----------[ Times:1 K:5 Type:odd N_size:<126 ]-----------
Test Loss: 0.02507, Test RMSE: 0.15832
-----------[ Times:1 K:6 Type:odd N_size:<126 ]-----------
Test Loss: 0.00961, Test RMSE: 0.09801
-----------[ Times:1 K:7 Type:odd N_size:<126 ]-----------
Test Loss: 0.02102, Test RMSE: 0.14497
-----------[ Times:1 K:8 Type:odd N_size:<126 ]-----------
Test Loss: 0.33542, Test RMSE: 0.57916
-----------[ Times:1 K:9 Type:odd N_size:<126 ]-----------
Test Loss: 0.14652, Test RMSE: 0.38278
-----------[ Times:1 K:10 Type:odd N_size:<126 ]-----------
Test Loss: 0.06055, Test RMSE: 0.24607
total rmse of k fol

RuntimeError: Error(s) in loading state_dict for FM_MLP:
	Missing key(s) in state_dict: "fnn1.0.weight", "fnn1.0.bias", "fnn1.2.weight", "fnn1.2.bias", "fnn1.2.running_mean", "fnn1.2.running_var", "fnn1.3.weight", "fnn1.3.bias". 
	Unexpected key(s) in state_dict: "encoder.0.weight", "encoder.0.bias", "encoder.2.weight", "encoder.2.bias", "encoder.2.running_mean", "encoder.2.running_var", "encoder.2.num_batches_tracked", "encoder.3.weight", "encoder.3.bias", "encoder.5.weight", "encoder.5.bias", "decoder.0.weight", "decoder.0.bias", "decoder.2.weight", "decoder.2.bias", "decoder.2.running_mean", "decoder.2.running_var", "decoder.2.num_batches_tracked", "decoder.3.weight", "decoder.3.bias", "decoder.5.weight", "decoder.5.bias", "predicter.0.weight", "predicter.0.bias", "predicter.2.weight", "predicter.2.bias", "predicter.2.running_mean", "predicter.2.running_var", "predicter.2.num_batches_tracked", "predicter.3.weight", "predicter.3.bias", "predicter.5.weight", "predicter.5.bias", "predi